In [ ]:
from tkinter import Tk, filedialog, Button, Label, Canvas
import matplotlib.pyplot as plt
from PIL import Image, ImageTk
import numpy as np
import os
import cv2


In [ ]:
# Function to load the image and convert to grayscale
def load_and_convert_to_grayscale(image_path):
    image = Image.open(image_path).convert("RGB")  # Load image as RGB
    grayscale = np.array(image).men(axis=2).astype(np.uint8)  # Convert to grayscale manually
    return np.array(image), grayscale

In [ ]:
# Function to apply Gaussian blur (manual implementation)
def apply_gaussian_blur(image, kernel_size=5, sigma=1.0):
    def gaussian_kernel(size, sigma):
        ax = np.arange(-(size // 2), size // 2 + 1)
        kernel = np.exp(-(ax ** 2) / (2 * sigma ** 2))
        kernel = kernel / kernel.sum()
        return kernel[:, None] @ kernel[None, :]  # Outer product for 2D kernel
    kernel = gaussian_kernel(kernel_size, sigma)
    padded_image = np.pad(image, kernel_size // 2, mode="edge")
    blurred_image = np.zeros_like(image)
    
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            region = padded_image[i:i + kernel_size, j:j + kernel_size]
            blurred_image[i, j] = np.sum(region * kernel)
    return blurred_image.astype(np.uint8)

In [ ]:
# Function to perform edge detection manually (Sobel operator)
def edge_detection(image):
    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    sobel_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
    
    gradient_x = np.zeros_like(image, dtype=np.float32)
    gradient_y = np.zeros_like(image, dtype=np.float32)
    padded_image = np.pad(image, 1, mode="edge")
    
    for i in range(1, image.shape[0] + 1):
        for j in range(1, image.shape[1] + 1):
            region = padded_image[i - 1:i + 2, j - 1:j + 2]
            gradient_x[i - 1, j - 1] = np.sum(region * sobel_x)
            gradient_y[i - 1, j - 1] = np.sum(region * sobel_y)
    
    gradient_magnitude = np.sqrt(gradient_x**2 + gradient_y**2)
    gradient_magnitude = (gradient_magnitude / gradient_magnitude.max() * 255).astype(np.uint8)
    return gradient_magnitude

In [ ]:
# Function to apply Laplacian filter
def laplacian_filter(image):
    kernel = np.array([[0, 1, 0],
                       [1, -4, 1],
                       [0, 1, 0]])  # Laplacian kernel
    padded_image = np.pad(image, 1, mode="edge")
    laplacian_image = np.zeros_like(image, dtype=np.float32)
    
    for i in range(1, image.shape[0] + 1):
        for j in range(1, image.shape[1] + 1):
            region = padded_image[i - 1:i + 2, j - 1:j + 2]
            laplacian_image[i - 1, j - 1] = np.sum(region * kernel)
    
    laplacian_image = np.clip(laplacian_image, 0, 255)  # Clip values to valid range
    return laplacian_image.astype(np.uint8)

In [ ]:
# Improved Connected Components Algorithm
def connected_components(binary_image):
    height, width = binary_image.shape
    labels = np.zeros((height, width), dtype=int)
    label = 1
    equivalences = {}

    # First pass: assign initial labels
    for i in range(height):
        for j in range(width):
            if binary_image[i, j] == 255:  # If the pixel is part of a region
                neighbors = []
                if i > 0 and labels[i - 1, j] > 0:  # Top neighbor
                    neighbors.append(labels[i - 1, j])
                if j > 0 and labels[i, j - 1] > 0:  # Left neighbor
                    neighbors.append(labels[i, j - 1])

                if not neighbors:  # No labeled neighbors
                    labels[i, j] = label
                    equivalences[label] = {label}
                    label += 1
                else:  # Assign the smallest neighbor label
                    smallest_label = min(neighbors)
                    labels[i, j] = smallest_label
                    for neighbor_label in neighbors:
                        equivalences[smallest_label].update(equivalences[neighbor_label])
                        equivalences[neighbor_label] = equivalences[smallest_label]

    # Second pass: resolve equivalences
    for i in range(height):
        for j in range(width):
            if labels[i, j] > 0:
                labels[i, j] = min(equivalences[labels[i, j]])

    # Count sizes of labeled regions
    unique_labels, counts = np.unique(labels, return_counts=True)
    label_map = {label: count for label, count in zip(unique_labels, counts) if label > 0}

    return labels, label_map

In [ ]:
# Adjust the threshold function to ensure better output
def threshold_image(image, threshold):
    binary_image = (image >= threshold).astype(np.uint8) * 255  # Binary threshold
    return binary_image

In [ ]:
# Morphological closing to fix fragmented regions
def morphological_closing(binary_image, kernel_size=5):
    kernel = np.ones((kernel_size, kernel_size), dtype=np.uint8)
    padded_image = np.pad(binary_image, kernel_size // 2, mode="constant")
    closed_image = np.zeros_like(binary_image)

    # Apply morphological closing
    for i in range(binary_image.shape[0]):
        for j in range(binary_image.shape[1]):
            region = padded_image[i:i + kernel_size, j:j + kernel_size]
            if np.sum(region) == 255 * kernel_size * kernel_size:  # Fully white kernel
                closed_image[i, j] = 255

    return closed_image

In [ ]:
# Circularity-based filtering
def filter_circular_regions(label_map, labeled_image):
    valid_labels = []
    for label, size in label_map.items():
        # Find pixels belonging to this label
        coordinates = np.argwhere(labeled_image == label)
        x_coords, y_coords = coordinates[:, 0], coordinates[:, 1]

        # Calculate bounding box dimensions
        x_min, x_max = x_coords.min(), x_coords.max()
        y_min, y_max = y_coords.min(), y_coords.max()
        bounding_box_area = (x_max - x_min + 1) * (y_max - y_min + 1)

        # Circularity metric (area-to-bounding-box ratio)
        if 0.7 <= size / bounding_box_area <= 1.0:  # Adjust thresholds as needed
            valid_labels.append(label)

    return set(valid_labels)

In [ ]:

# Fonction pour charger et segmenter l'image
def segmenter_image():
    # Ouvrir une fenêtre pour sélectionner l'image
    chemin_image = filedialog.askopenfilename(title="Sélectionner une image", filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    
    # Charger l'image
    image = cv2.imread(chemin_image)
    gris = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convertir en niveaux de gris
    
    # Appliquer un flou pour réduire le bruit
    flou = cv2.GaussianBlur(gris, (5, 5), 0)
    
    # Seuil pour binariser l'image
    _, seuil = cv2.threshold(flou, 127, 255, cv2.THRESH_BINARY_INV)
    
    # Trouver les contours (segments)
    contours, _ = cv2.findContours(seuil, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Dessiner les contours sur l'image originale
    image_segmentee = image.copy()
    cv2.drawContours(image_segmentee, contours, -1, (0, 255, 0), 2)
    
    # Compter le nombre de pièces
    nombre_pieces = len(contours)
    
    # Afficher l'image segmentée
    afficher_image_segmentee(image_segmentee)
    
    # Afficher le nombre de pièces détectées
    label_nombre_pieces.config(text=f"Nombre de pièces : {nombre_pieces}")

# Fonction pour afficher l'image segmentée dans l'interface
def afficher_image_segmentee(image_segmentee):
    # Convertir l'image de OpenCV (BGR) à PIL (RGB)
    image_segmentee_rgb = cv2.cvtColor(image_segmentee, cv2.COLOR_BGR2RGB)
    image_pil = Image.fromarray(image_segmentee_rgb)
    
    # Convertir l'image PIL en format compatible avec Tkinter
    image_tk = ImageTk.PhotoImage(image_pil)
    
    # Afficher l'image sur le canevas
    canvas.create_image(0, 0, anchor='nw', image=image_tk)
    canvas.image = image_tk

# Créer la fenêtre principale
fenetre = Tk()
fenetre.title("Segmentation d'image")

# Créer un bouton pour charger et segmenter l'image
bouton_segmenter = Button(fenetre, text="Charger et segmenter une image", command=segmenter_image)
bouton_segmenter.pack()

# Créer un label pour afficher le nombre de pièces
label_nombre_pieces = Label(fenetre, text="Nombre de pièces : 0")
label_nombre_pieces.pack()

# Créer un canevas pour afficher l'image
canvas = Canvas(fenetre, width=600, height=600)
canvas.pack()
os.environ['TK_SILENCE_DEPRECATION'] = '1'
# Démarrer l'interface graphique
fenetre.mainloop()

In [ ]:
# Updated Main Pipeline
def main(image_path):
    # Step 1: Load and preprocess the image
    original_image, grayscale_image = load_and_convert_to_grayscale(image_path)
    blurred_image = apply_gaussian_blur(grayscale_image)
    
    # Step 2: Apply edge detection using Sobel and Laplacian
    sobel_edges = edge_detection(blurred_image)
    laplacian_edges = laplacian_filter(blurred_image)
    
    # Combine Sobel and Laplacian edges
    combined_edges = np.maximum(sobel_edges, laplacian_edges)
    
    # Step 3: Apply thresholding
    thresholded_image = threshold_image(combined_edges, threshold=50)
    
    # Step 4: Morphological closing to reduce over-segmentation
    closed_image = morphological_closing(thresholded_image)
    
    # Step 5: Count coins using connected components
    labeled_image, label_map = connected_components(closed_image)
    
    # Step 6: Filter by size and circularity
    valid_labels = filter_circular_regions(label_map, labeled_image)
    coin_count = len(valid_labels)
    
    print(f"Total coins detected: {coin_count}")
    
    # Visualization
    plt.figure(figsize=(15, 6))
    plt.subplot(1, 6, 1)
    plt.title("Original Image")
    plt.imshow(original_image)
    
    plt.subplot(1, 6, 2)
    plt.title("Grayscale Image")
    plt.imshow(grayscale_image, cmap="gray")
    
    plt.subplot(1, 6, 3)
    plt.title("Sobel + Laplacian Edges")
    plt.imshow(combined_edges, cmap="gray")
    
    plt.subplot(1, 6, 4)
    plt.title("Thresholded Image")
    plt.imshow(thresholded_image, cmap="gray")
    
    plt.subplot(1, 6, 5)
    plt.title("Closed Image")
    plt.imshow(closed_image, cmap="gray")
    
    plt.subplot(1, 6, 6)
    plt.title("Labeled Coins")
    plt.imshow(np.isin(labeled_image, list(valid_labels)), cmap="nipy_spectral")
    
    plt.show()

image_path = "../coin-counting/dataset/coins_images/coins_images/all_coins/0c7067ece2.jpg"  # Replace with your image path
main(image_path)